# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests
from pathlib import Path

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
csv_file = Path("output_data/cities.csv")
city_data_df = pd.read_csv(csv_file)

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,adamstown,-25.0660,-130.1015,22.14,76,86,4.65,PN,1715300910
1,1,huaibei,33.9744,116.7917,19.67,58,13,4.04,CN,1715300910
2,2,bilibino,68.0546,166.4372,-0.81,88,100,0.92,RU,1715300911
3,3,ust-nera,64.5667,143.2000,3.59,83,100,1.78,RU,1715300911
4,4,puerto natales,-51.7236,-72.4875,0.51,93,99,1.31,CL,1715300912


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [16]:
%%capture --no-display

map_plot = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 700,
    frame_height = 500,
    size = "Humidity",
    scale = 1.5,
    color = "City"
)

# Display the map plot
map_plot

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [4]:
# Narrow down cities that fit criteria and drop any results with null values
city_data_df

my_ideal_weather_df = city_data_df[(city_data_df['Max Temp'] > 21) & (city_data_df['Max Temp'] < 27) & (city_data_df['Wind Speed'] < 4.5) & (city_data_df['Cloudiness'] == 0)]

# Drop any rows with null values
my_ideal_weather_cleaned_df = my_ideal_weather_df.dropna()

# Display sample data
my_ideal_weather_cleaned_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
33,33,conde,-7.2597,-34.9075,24.70,94,0,1.03,BR,1715300928
51,51,belmonte,-15.8631,-38.8828,25.29,83,0,1.70,BR,1715300938
98,98,arraial do cabo,-22.9661,-42.0278,22.98,88,0,0.51,BR,1715300964
130,130,pisco,-13.7000,-76.2167,21.12,88,0,2.57,PE,1715300982
139,139,coahuayana de hidalgo,18.7000,-103.6583,26.91,62,0,2.61,MX,1715300987


### Step 3: Create a new DataFrame called `hotel_df`.

In [5]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = my_ideal_weather_cleaned_df[['City', 'Country', 'Lat','Lng','Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
33,conde,BR,-7.2597,-34.9075,94,
51,belmonte,BR,-15.8631,-38.8828,83,
98,arraial do cabo,BR,-22.9661,-42.0278,88,
130,pisco,PE,-13.7000,-76.2167,88,
139,coahuayana de hidalgo,MX,18.7000,-103.6583,62,
169,benjamin constant,BR,-4.3831,-70.0311,88,
208,upington,ZA,-28.4478,21.2561,33,
260,ipora,BR,-16.4419,-51.1178,60,
269,greenville,US,35.6127,-77.3663,79,
274,paulden,US,34.8856,-112.4682,13,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [7]:
# Set parameters to search for a hotel
radius = 10000
params = {
    "categories":"accommodation.hotel",
    "apiKey": geoapify_key,
    "limit":20
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():

    # Get the city's name
    city = row["City"]
    
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]
    
    # Add filter and bias parameters with the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"
    
    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"


    # Make an API request using the params dictionaty
    name_address = requests.get(base_url, params=params)

    
    # Convert the API response to JSON format
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"
        
    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
conde - nearest hotel: Paladino
belmonte - nearest hotel: No hotel found
arraial do cabo - nearest hotel: No hotel found
pisco - nearest hotel: La Portada
coahuayana de hidalgo - nearest hotel: No hotel found
benjamin constant - nearest hotel: No hotel found
upington - nearest hotel: Die Eiland Holiday Ressort
ipora - nearest hotel: No hotel found
greenville - nearest hotel: Quality Inn
paulden - nearest hotel: No hotel found
nova vicosa - nearest hotel: Pousada Pontal da Barra
turpan - nearest hotel: Oriental
ghat - nearest hotel: فندق تاسيلي
ixtapa - nearest hotel: Ma. Cristina


,City,Country,Lat,Lng,Humidity,Hotel Name
33,conde,BR,-7.2597,-34.9075,94,Paladino
51,belmonte,BR,-15.8631,-38.8828,83,No hotel found
98,arraial do cabo,BR,-22.9661,-42.0278,88,No hotel found
130,pisco,PE,-13.7000,-76.2167,88,La Portada
139,coahuayana de hidalgo,MX,18.7000,-103.6583,62,No hotel found
169,benjamin constant,BR,-4.3831,-70.0311,88,No hotel found
208,upington,ZA,-28.4478,21.2561,33,Die Eiland Holiday Ressort
260,ipora,BR,-16.4419,-51.1178,60,No hotel found
269,greenville,US,35.6127,-77.3663,79,Quality Inn
274,paulden,US,34.8856,-112.4682,13,No hotel found


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [14]:
%%capture --no-display

# Configure the map plot
map_plot_hotels = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 500,
    scale = 2.0,
    size = 100.0,
    color = "City",
    hover_cols =['Humidity','Hotel Name','Country']
    
)

# Display the map
map_plot_hotels

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Hotel Name,Country)